# Pandas Dataframe

本 Notebook 展示了如何使用 Agent 与 `Pandas DataFrame` 进行交互。它主要针对问答进行了优化。

**注意：此 Agent 在底层调用 `Python` Agent，该 Agent 会执行 LLM 生成的 Python 代码——如果 LLM 生成的 Python 代码有害，可能会带来风险。请谨慎使用。**

**注意：由于 langchain 已迁移到 v0.3，您应该升级 langchain_openai 和 langchain。这将避免导入错误。**

pip install --upgrade langchain_openai
pip install --upgrade langchain

In [1]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [2]:
import pandas as pd
from langchain_openai import OpenAI

df = pd.read_csv(
    "https://raw.githubusercontent.com/pandas-dev/pandas/main/doc/data/titanic.csv"
)

## 使用 `ZERO_SHOT_REACT_DESCRIPTION`

这展示了如何使用 `ZERO_SHOT_REACT_DESCRIPTION` 代理类型来初始化代理。

In [3]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

## 使用 OpenAI Functions

这展示了如何使用 OPENAI_FUNCTIONS 代理类型来初始化代理。请注意，这是上述方法的替代方案。

In [4]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [5]:
agent.invoke("how many rows are there?")



> Entering new  chain...

Invoking: `python_repl_ast` with `df.shape[0]`


891There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'

In [5]:
agent.invoke("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people with more than 3 siblings
Action: python_repl_ast
Action Input: df[df['SibSp'] > 3].shape[0]
Observation: 30
Thought: I now know the final answer
Final Answer: 30 people have more than 3 siblings.

> Finished chain.


'30 people have more than 3 siblings.'

In [6]:
agent.invoke("whats the square root of the average age?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the average age first
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 29.69911764705882
Thought: I now need to calculate the square root of the average age
Action: python_repl_ast
Action Input: math.sqrt(df['Age'].mean())
Observation: NameError("name 'math' is not defined")
Thought: I need to import the math library
Action: python_repl_ast
Action Input: import math
Observation: 
Thought: I now need to calculate the square root of the average age
Action: python_repl_ast
Action Input: math.sqrt(df['Age'].mean())
Observation: 5.449689683556195
Thought: I now know the final answer
Final Answer: The square root of the average age is 5.449689683556195.

> Finished chain.


'The square root of the average age is 5.449689683556195.'

## 多 DataFrame 示例

接下来这部分展示了代理如何与作为列表传入的多个 DataFrame 进行交互。

In [7]:
df1 = df.copy()
df1["Age"] = df1["Age"].fillna(df1["Age"].mean())

In [8]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), [df, df1], verbose=True)
agent.invoke("how many rows in the age column are different?")



> Entering new AgentExecutor chain...
Thought: I need to compare the age columns in both dataframes
Action: python_repl_ast
Action Input: len(df1[df1['Age'] != df2['Age']])
Observation: 177
Thought: I now know the final answer
Final Answer: 177 rows in the age column are different.

> Finished chain.


'177 rows in the age column are different.'